In [332]:
import csv
import json
import time
import heapq
from datetime import datetime, timedelta
from math import sqrt
import math
import random

In [333]:
def read_csv_to_matrix(file_path):
    data_matrix = []

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        for row in csv_reader:
            data_matrix.append(row)

    return data_matrix

csv_file_path = 'drivers.csv'
drivers_matrix = read_csv_to_matrix('drivers.csv')
passengers_matrix = read_csv_to_matrix('passengers.csv')
edges_matrix = read_csv_to_matrix('edges.csv')

In [334]:
print("Drivers")
for i, row in enumerate(drivers_matrix[:5]):
    print(row)
print("Passengers")
for i, row in enumerate(passengers_matrix[:5]):
    print(row)
print("Edges")
for i, row in enumerate(edges_matrix[:5]):
    print(row)

Drivers
['Date/Time', 'Source Lat', 'Source Lon']
['04/25/2014 00:14:00', '40.667', '-73.8713']
['04/25/2014 00:21:00', '40.6482', '-73.7831']
['04/25/2014 00:31:00', '40.7648', '-73.996']
['04/25/2014 00:37:00', '40.74', '-73.999']
Passengers
['Date/Time', 'Source Lat', 'Source Lon', 'Dest Lat', 'Dest Lon']
['04/25/2014 00:00:00', '40.6466', '-73.7896', '40.7603', '-73.9794']
['04/25/2014 00:07:00', '40.7155', '-73.9933', '40.7205', '-73.9872']
['04/25/2014 00:08:00', '40.7157', '-74.0064', '40.7064', '-74.0167']
['04/25/2014 00:10:00', '40.7523', '-73.9953', '40.7563', '-73.9915']
Edges
['start_id', 'end_id', 'length', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'weekday_8', 'weekday_9', 'weekday_10', 'weekday_11', 'weekday_12', 'weekday_13', 'weekday_14', 'weekday_15', 'weekday_16', 'weekday_17', 'weekday_18', 'weekday_19', 'weekday_20', 'weekday_21', 'weekday_22', 'weekday_23', 'weekend_0', 'weekend_1', 'weekend_2', 'weeke

In [335]:
def read_json_to_dict(file_path):
    with open(file_path, 'r') as json_file:
        data_dict = json.load(json_file)

    return data_dict

json_file_path = 'node_data.json'
node_data_dict = read_json_to_dict(json_file_path)

count = 0
for node_id, coordinates in node_data_dict.items():
    #print(f'Node ID: {node_id}, Coordinates: {coordinates}')
    count += 1
    if count == 5:
        break

# T1 
- Create quques to keep track of available drivers and passengers 
- When a driver is free, assign longest waiting passenger to them (the one with the earliest date/time)
- Disregard distances
- When matched, remove driver and passenger from the queue
- update to recycle drivers (find a way to estimate how long each ride takes-- use the graph?)

In [336]:
def match_passenger_to_driver(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        current_passenger_time, current_passenger = heapq.heappop(passenger_heap)
        # remove conditional, assign first passenger to first driver
        #print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver(drivers_matrix[1:], passengers_matrix[1:])

Matching process complete. Elapsed time: 0.10879302024841309 seconds


# T2
### Consider a slightly improved baseline that, whenever there is a choice, schedules the driver-passenger pair with minimum straight-line distance between the driver and pickup location of the passenger. Implement this algorithm and conduct an experiment measuring the performance of the algorithm in terms of desiderata D1-D3.
- Use Euclidean distance 
- 

In [337]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Calculate the Euclidean distance between two sets of coordinates
    dx = lon2 - lon1
    dy = lat2 - lat1
    distance = sqrt(dx**2 + dy**2)
    return distance

def match_passenger_to_driver_t2(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    # heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        dist = float('inf')
        current_passenger = 0
        #count = -1
        #current_passenger_ind = 0
        for passenger in passenger_heap:
            #count = count + 1
            if passenger[0] < current_driver_time:
                if euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2])) < dist:
                    dist = euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2]))
                    current_passenger = passenger
                    #current_passenger_ind = count
        if current_passenger != 0:
            passenger_heap.remove(current_passenger)
        # remove conditional, assign first passenger to first driver
            #print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger[1]}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver_t2(drivers_matrix[1:], passengers_matrix[1:])

Matching process complete. Elapsed time: 2.595546007156372 seconds


## PreProcessing the roade network graph

### creating a node dictionary, an edges dictionary, and an adjacency dictionary

Graph Nodes:
Node ID: 42467330, Node Data: {'lon': -73.933676, 'lat': 40.655156}
Node ID: 42467331, Node Data: {'lon': -73.932706, 'lat': 40.655216}
Node ID: 42467333, Node Data: {'lon': -73.931772, 'lat': 40.655273}
Node ID: 42467334, Node Data: {'lon': -73.930768, 'lat': 40.655336}
Node ID: 42467335, Node Data: {'lon': -73.929762, 'lat': 40.655398}

Graph Edges:
Edge: (39076461, 42854803), Attributes: {'length': 0.1582682216504902, 'weekday_hours': [19.8665, 19.986, 17.5154094776635,..], 'weekend_hours': [2, 3, 4...]}

In [338]:
def read_graph_nodes_and_edges():
    with open('node_data.json') as nodes_file:
        nodes = json.load(nodes_file)

    edges = {}
    with open('edges.csv', 'r') as edges_csv:
        edges_reader = csv.DictReader(edges_csv)
        for row in edges_reader:
            start_id = int(row['start_id'])
            end_id = int(row['end_id'])
            length = float(row['length'])

            # Additional attributes for each hour of the day
            weekday_hours = [float(row[f'weekday_{i}']) for i in range(24)]
            weekend_hours = [float(row[f'weekend_{i}']) for i in range(24)]

            edge = (start_id, end_id)

            edges[edge] = {
                'length': length,
                'weekday_hours': weekday_hours,
                'weekend_hours': weekend_hours
            }

    return nodes, edges
nodes, edges = read_graph_nodes_and_edges()

# Display the result
print("Graph Nodes:")

for node_id, node_data in list(nodes.items())[:5]:
    print(f'Node ID: {node_id}, Node Data: {node_data}')

print("\nGraph Edges:")
for edge, attributes in list(edges.items())[:5]:
    print(f'Edge: {edge}, Attributes: {attributes}')

Graph Nodes:
Node ID: 42467330, Node Data: {'lon': -73.933676, 'lat': 40.655156}
Node ID: 42467331, Node Data: {'lon': -73.932706, 'lat': 40.655216}
Node ID: 42467333, Node Data: {'lon': -73.931772, 'lat': 40.655273}
Node ID: 42467334, Node Data: {'lon': -73.930768, 'lat': 40.655336}
Node ID: 42467335, Node Data: {'lon': -73.929762, 'lat': 40.655398}

Graph Edges:
Edge: (39076461, 42854803), Attributes: {'length': 0.1582682216504902, 'weekday_hours': [19.8665, 19.986, 17.5154094776635, 23.146191909356634, 28.761, 22.698500000000003, 20.252, 15.733307692307692, 17.98772222222222, 20.90642857142857, 16.653230769230767, 15.671076923076924, 15.118333333333332, 14.84723076923077, 10.344733333333332, 6.3204, 12.116307692307691, 11.693833333333332, 13.2156, 16.782888888888888, 13.669571428571428, 15.879916666666666, 17.7299, 17.159000000000002], 'weekend_hours': [17.1085, 18.66775, 21.257666666666665, 21.91733333333333, 20.2505, 16.061, 21.858666666666664, 19.485, 15.53875, 19.924750000000003

# Optimization if road network

Create clusters of nodes based on coordinate locations and connect them via edges. 
- The orignal graph has over 54128 nodes and 179140 edges
- For a sparse graph of 8,019 nodes, we created clusters based on coordinates with the latitude truncated to 2 decimal points and the longitutde truncated to 3 decimal points 
- We chose these values because it gave a reasonable reduction 
- reducing both latitude and longitude to three would still give us 34,000 clusters and reducing both to 3 would reduce down to 900 clusters.
- From the clusters, we create an adjacency dictionary that is indexed by every cluster. The value is a dictionary of all adjacent clusters and the average time it takes to traverse to that cluster (this is based on all edges and the speeds of all hours between the two clusters)
- Example key, value pair
(40.79, -73.795) {(40.79, -73.792): 0.009400790831058076, (40.79, -73.804): 0.02665183346506693, (40.79, -73.786): 0.02624477279791465, (40.79, -73.796): 0.002410152439330088, (40.79, -73.794): 0.003181105704966562, (40.79, -73.795): 0.003844184044364148, (40.78, -73.795): 0.005251628822716274, (40.79, -73.791): 0.009741447042992716, (40.79, -73.797): 0.0051788418127648355}


In [ ]:
# Create clusters graph 
def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 3)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)

def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 3))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 3))

        #print(cluster_key1,cluster_key2)
        # check if source and target are in different clusters aka they have different cluster keys 
        # if they are different, add it to the graph as an edge
        if cluster_key1 != cluster_key2:
            length = attributes["length"]
            speeds = attributes['weekday_hours']
            average_speed = sum(speeds)/len(speeds)
            estimated_time = length / average_speed
            # add this edge to the adjacency
            # Add source to target with weight
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(estimated_time)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [estimated_time]
        else:
            adjacency_dict[cluster_key1] = {cluster_key2: [estimated_time]}

        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency = update_edges_between_clusters(clusters, nodes)


for key1, nested_dict in adjacency.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            avg_list = sum(value) / len(value) 
            
            # Update the nested dictionary with the average list
            adjacency[key1][key2] = avg_list

print(len(adjacency))
for key, value in list(adjacency.items())[:5]:
    print(key, value)

In [ ]:
for key, value in list(clusters.items())[:5]:
    print(key, value)
    print(type(key[0]))

In [ ]:
# dictionary hashed by coordinates instead of node id -- dont think this is used 
coordinates_dict = {}

for node_id, coordinates in nodes.items():
    lon = coordinates['lon']
    lat = coordinates['lat']
    key = (lon, lat)
    coordinates_dict[key] = node_id

for node in list(coordinates_dict.items())[:5]:
    print(node)

# T3
### Consider a further improvement that, whenever there is a choice, schedules the driver-passenger pair with minimum estimated time for the driver to traverse the network to the passenger pickup location. Explain how you will do this algorithmically. Then implement this algorithm and conduct an experiment measuring the performance of the algorithm in terms of desiderata D1-D3. Your implementation does not have to be optimized for efficiency (see T4 below).

- Using Dijkstra's algorithm on the sparse road network
- Prioirty queues based on wait time 
- run dijkstras between each driver and passenger to find closest passenger to driver (approximately)
- find_closest_coordinates returns the closest cluster that the driver or passenger is near


In [ ]:
def dijkstras(graph, start, target):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    priority_queue = [(0, start)]
    heapq.heapify(priority_queue)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == target:
            break

        if current_distance <= distances[current_node]:
            for neighbor, weight in graph[(current_node)].items():
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))

    return distances[(target)]

def average_edge_distance(known_coordinates, graph):
    total_distance = 0
    count = 0

    for node1, neighbors in graph.items():
        for node2 in neighbors:
            print(node1)
            print(node2)
            distance = euclidean_distance(float(node1[0]), float(node1[1]), float(node2[0]), float(node2[1]))
            total_distance += distance
            count += 1

    if count == 0:
        return 0  # Avoid division by zero

    average_distance = total_distance / count
    return average_distance

def find_closest_coordinates(target_coordinates, known_coordinates, avgdist):
    min_distance = float('inf')
    closest_coordinates = None

    for key, value in known_coordinates.items():
        distance = euclidean_distance(target_coordinates[0], target_coordinates[1], key[0], key[1])
        """ if distance < avgdist*2:
            return key """
        if distance < min_distance:
            min_distance = distance
            closest_coordinates = key

    return closest_coordinates

def match_passenger_to_driver_t3(drivers, passengers, adjacency, clusters):
    start_time = time.time()

    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)

    count  = 0
    avg_dist = average_edge_distance(clusters, adjacency)
    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        shortest_path_time = float('inf')
        current_passenger = None
        current_passenger_time = None
        count1 = 0
        for passenger_time, passenger in passenger_heap:
            minni = timedelta(minutes=7)
            max_wait_time = current_driver_time - minni
            if passenger_time < current_driver_time and passenger_time > max_wait_time:
                #print((float(current_driver[1]), float(current_driver[2])))
                # Get the node IDs from the latitude and longitude using the Nodes dictionary
                s1_time = time.time()
                print(f"Start")
                start_node = find_closest_coordinates((float(current_driver[1]), float(current_driver[2])), clusters, avg_dist)
                end_node = find_closest_coordinates((float(passenger[1]), float(passenger[2])), clusters, avg_dist)
                print(f"closest coord done")
                e1_time = time.time()
                d1_time = e1_time - s1_time
                print(d1_time)
                print(f"--\n--\n--\n")
                print(f"Start")
                s_time = time.time()
                path_time = dijkstras(adjacency, start_node, end_node)
                print(f"Dijkstras done")
                count1 = count1 + 1
                e_time = time.time()
                d_time = e_time - s_time
                print(d_time)
                print(count1)
                print(f"--\n--\n--\n")
                if path_time < shortest_path_time:
                    shortest_path_time = path_time
                    current_passenger = passenger
                    current_passenger_time = passenger_time
        
        if current_passenger:
            print(current_passenger)
            passenger_heap.remove((current_passenger_time,current_passenger))
            current_driver[1] = current_passenger[3]
            current_driver[2] = current_passenger[4]
            count = count + 1
            print(count)
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")

            # Introduce a 95% chance for the driver to be pushed back
        if random.random() < 0.95:
            if shortest_path_time < 10000:
                drive_duration = timedelta(hours=shortest_path_time)
            else:
                drive_duration = timedelta(minutes=2)
            heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
match_passenger_to_driver_t3(drivers_matrix[1:], passengers_matrix[1:], adjacency, clusters)

# T4
Improve the runtime efficiency while maintaining the same general scheduling strategy as in T3. To accomplish this, consider the following two optimizations:
(i) Explain how to preprocess the nodes of the road network so that you can efficiently
find the closest (in terms of straight-line distance) node to a given query point (at least
approximately). By efficient, your algorithm should scale sub-linearly with |V | (i.e., it
should not simply loop over all nodes of the road network).
(ii) Describe an algorithm to compute shortest paths between two nodes of the road network
in a way that is more efficient than just Dijkstra’s algorithm. Again, you may consider
possible pre-processings, and you might consider computing approximate rather than
exact shortest paths. Keep in mind that speed on road segments may vary by day and
time.
Conduct experiments and provide evidence that both of your optimizations are at least approximately correct empirically. Also conduct an experiment to measure the the performance
of the algorithm in terms of desiderata D1-D3, particularly with a focus on the improvement
in query runtimes over the strategy from T3.


- For Floyd warshall which runs in exponential time, we are making the graph sparser by shortening the coordinates to 2 decimal places. Now, we have 949 clusters

In [339]:
# Create a new clusters graph with fewer nodes for floyd warshall
def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 2)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)
print(clusters)
def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 2))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 2))

        #print(cluster_key1,cluster_key2)
        # if the cluster keys are the same, still add it to the graph. 
        # if they are different, add it to the graph as an edge

        length = attributes["length"]
        speeds = attributes['weekday_hours']
        average_speed = sum(speeds)/len(speeds)
        estimated_time = length / average_speed
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(estimated_time)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [estimated_time]
      
        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency = update_edges_between_clusters(clusters, nodes)

for key1, nested_dict in adjacency.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            avg_list = sum(value) / len(value) 
            
            # Update the nested dictionary with the average list
            adjacency[key1][key2] = avg_list

print(len(adjacency))
for key, value in list(adjacency.items())[:5]:
    print(key, value)

{(40.66, -73.93): ['42467330', '42467331', '42467333', '42467334', '42467335', '42467337', '42467339', '42467343', '42467346', '42467776', '42467777', '42468550', '42468552', '42468557', '42468562', '42468566', '42468569', '42468570', '42468573', '42468574', '42468576', '42468577', '3534079545', '42472185', '42472191', '42472195', '42473144', '42473147', '42479643', '42479645', '42479646', '42481987', '42485107', '42485112', '42485115', '42489551', '42514764', '42494693', '42494695', '42494701', '42494706', '42494709', '42494712', '42494716', '42494719', '42497488', '42497490', '42497493', '42497496', '42497535', '42497547', '42497552', '42497556', '42497560', '42516395', '42500152', '42500183', '42500184', '42503266', '4749827004', '42513375', '42513385', '42514692', '42514697', '42514702', '42514710', '42514760', '42497498', '42520965', '42522002', '42522034', '42522040', '42522051', '42522054', '4929411563', '4375241836', '4207995533', '4375241842', '3547815597', '3654919263', '4246

In [341]:
def floyd_warshall(graph):
    vertices = list(graph.keys())

    # Initialize the distance matrix and the dictionary for intermediate vertices
    dist = []
    nIndex = {}
    
    # Create the nIndex using a regular for loop
    for index, key in enumerate(vertices):
        nIndex[key] = index
    
    # Initialize the distance matrix
    #counte = 0
    for i in vertices:
        row = []
        for j in vertices:
            if i == j:
                row.append(0)
            elif j in graph[i]:
                row.append(graph[i][j]) # the new adajcency graph will have to contain a dictionary or list of speeds/times
            else:
                row.append(float('inf'))
        dist.append(row)
        #counte = counte + 1
        #print(counte)

    # Rest of the Floyd-Warshall algorithm
    for k in vertices:
        print(f"flooooooooooyd MAYYYYWEATHHHHHERRRRRRRRR")
        for i in vertices:
            #print(f"MIDDLEEEEeeEEEEEE")
            for j in vertices:
                #print(f"inner loop")
                # add another for loop for each comparison for each speed. Each cell of the 2d matrix will contain a list/dictionary for each day of the week. 
                # When we look up distances, we can look up by week
                if dist[nIndex[i]][nIndex[k]] + dist[nIndex[k]][nIndex[j]] < dist[nIndex[i]][nIndex[j]]:
                    dist[nIndex[i]][nIndex[j]] = dist[nIndex[i]][nIndex[k]] + dist[nIndex[k]][nIndex[j]]

    return dist, nIndex


In [ ]:
# Floyd warshall using average speeds (for all days of the week between clusters)
ftime = time.time()
adjacency1 , indices = floyd_warshall(adjacency)
et = time.time()
elapsed_time = et - ftime
print(f"Floyd Warshall creation Elapsed time: {elapsed_time} seconds")
print(f"YAYYYYYY!!!!!!!")

In [343]:
# closest coordinates using clusters instead of nodes. Returns closest cluster coordinates
def find_closest_coordinates(target_coordinates, known_coordinates):
    min_distance = float('inf')
    closest_coordinates = None

    for key, value in known_coordinates.items():
        distance = euclidean_distance(target_coordinates[0], target_coordinates[1], key[0], key[1])
        if distance < min_distance:
            min_distance = distance
            closest_coordinates = key

    return closest_coordinates

def match_passenger_to_driver_t4(drivers, passengers, adjacency_matrices, nodes, nIndex):
    start_time = time.time()

    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)

    count = 0

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        shortest_path_time = float('inf')
        current_passenger = None
        current_passenger_time = None

        for passenger_time, passenger in passenger_heap:
            minni = timedelta(minutes=7)
            max_wait_time = current_driver_time - minni

            if passenger_time < current_driver_time and passenger_time > max_wait_time:
                start_node = find_closest_coordinates((float(current_driver[1]), float(current_driver[2])), nodes)
                end_node = find_closest_coordinates((float(passenger[1]), float(passenger[2])), nodes)
                path_time = adjacency_matrices[nIndex[start_node]][nIndex[end_node]]
                
                if path_time < shortest_path_time:
                    shortest_path_time = path_time
                    current_passenger = passenger
                    current_passenger_time = passenger_time

        if current_passenger:
            print(current_passenger)
            passenger_heap.remove((current_passenger_time, current_passenger))
            current_driver[1] = current_passenger[3]
            current_driver[2] = current_passenger[4]
            count = count + 1
            print(count)
            #print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")

            # Introduce a 95% chance for the driver to be pushed back
        if random.random() < 0.95:
            if shortest_path_time < 10000:
                drive_duration = timedelta(hours=shortest_path_time)
            else:
                drive_duration = timedelta(minutes=2)
            heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")

In [ ]:
match_passenger_to_driver_t4(drivers_matrix[1:], passengers_matrix[1:], adjacency1, clusters, indices)

# T5 
Describe and implement your own alternative scheduling algorithm to
improve over the baseline algorithms. The improvement might be on any one or multiple
of the desiderata D1-D3, or on some other important desirable properties that you propose.
Conduct experiments to demonstrate the improvement.

- We are increasing the accuracy of the path estimation by including speeds depending on the time of day and day of the week 
- We create a new floyd warshall matrix that includes the shortest times bewteen nodes depending on the hour and day of the week 
- In the matching algorithm, we maximize the ride profit: Drivers want to maximize ride profit, defined as the number of minutes they spend
driving passengers from pickup to drop-off locations minus the number of minutes they spend
driving to pickup passengers. For simplicity, there is no penalty for the time when a driver is
idle
- We choose the passenger that will maximize the time it takes between these locations: (passenger start - passenger stop) - (driver loc - passenger start)
- if these locations are within nodes, we will use the average time it takes for the hour of the day 

In [347]:
# Use th esame clusters as before, update adjacency to include all speeds
""" def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 2)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)
print(clusters) """
def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 2))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 2))

        #print(cluster_key1,cluster_key2)
        # if the cluster keys are the same, still add it to the graph. 
        # if they are different, add it to the graph as an edge

        length = attributes["length"]
        speeds = attributes['weekday_hours'] + attributes['weekend_hours']

        # estimated times
        
        times = list(map(lambda x: length / x, speeds))

        #average_speed = sum(speeds)/len(speeds)
        #estimated_time = length / average_speed
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(times)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [times]
      
        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency2 = update_edges_between_clusters(clusters, nodes)

print(len(adjacency2))
# Each edge now has a list of length 48 that contains the time it takes to travel along an edge for each hour of the day
for key1, nested_dict in adjacency2.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            # Use zip to group elements at the same index
            zipped_lists = zip(*value)

            # Calculate the average for each group of elements
            averages = [sum(values) / len(values) for values in zipped_lists]
            
            # Update the nested dictionary with the average list
            adjacency2[key1][key2] = averages

for key, value in list(adjacency2.items())[:5]:
    print(key, value)

949
(40.79, -73.79) {(40.79, -73.8): [[0.02414998525648045, 0.024357207247852418, 0.02353208053251991, 0.022031463160768715, 0.018852465558771906, 0.018916950980978255, 0.025313438792394154, 0.026354024068112637, 0.02966299110249242, 0.028252471070847177, 0.03504969887311731, 0.032189310477581994, 0.02786152914607015, 0.035183846033572476, 0.037325803562243395, 0.031473696201502324, 0.030957465871819117, 0.02663809238341535, 0.02707870992828138, 0.026533103509613687, 0.027304225113513672, 0.028648542607853735, 0.025789121843655366, 0.02472307518112695, 0.024206869365231937, 0.02224020989565361, 0.022097106736157546, 0.024678830237031785, 0.02297064839280155, 0.02139965084633803, 0.02449884198941718, 0.02215708583656813, 0.02412180620118563, 0.02266259604703756, 0.023026371439066717, 0.024263263496653482, 0.02603150602975871, 0.022708018904935828, 0.03170243460262015, 0.027795121264589474, 0.027506929768937618, 0.03386798545942468, 0.030365862210902844, 0.026362686668494863, 0.025754553